In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('/content/drive/My Drive/Python/TEMP/data/train.csv', index_col=0)
print(train.head())
print()
print(train.dtypes)

             date      speed
id                          
0   1/1/2017 0:00  43.002930
1   1/1/2017 1:00  46.118696
2   1/1/2017 2:00  44.294158
3   1/1/2017 3:00  41.067468
4   1/1/2017 4:00  46.448653

date      object
speed    float64
dtype: object


In [3]:
train['date'] =  pd.to_datetime(train['date'], format='%d/%m/%Y %H:%M')
print(train.head())
print()
print(train.dtypes)

                  date      speed
id                               
0  2017-01-01 00:00:00  43.002930
1  2017-01-01 01:00:00  46.118696
2  2017-01-01 02:00:00  44.294158
3  2017-01-01 03:00:00  41.067468
4  2017-01-01 04:00:00  46.448653

date     datetime64[ns]
speed           float64
dtype: object


In [4]:
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train['weekday'] = train['date'].dt.dayofweek
train['hour'] = train['date'].dt.hour
print(train.head())
print()
print(train.dtypes)

                  date      speed  year  month  day  weekday  hour
id                                                                
0  2017-01-01 00:00:00  43.002930  2017      1    1        6     0
1  2017-01-01 01:00:00  46.118696  2017      1    1        6     1
2  2017-01-01 02:00:00  44.294158  2017      1    1        6     2
3  2017-01-01 03:00:00  41.067468  2017      1    1        6     3
4  2017-01-01 04:00:00  46.448653  2017      1    1        6     4

date       datetime64[ns]
speed             float64
year                int64
month               int64
day                 int64
weekday             int64
hour                int64
dtype: object


In [5]:
holiday_2017 = pd.read_csv('/content/drive/My Drive/Python/TEMP/data/holiday_2017.csv')
holiday_2018 = pd.read_csv('/content/drive/My Drive/Python/TEMP/data/holiday_2018.csv')
holiday_2017['date'] =  pd.to_datetime(holiday_2017['date'], format='%d/%m/%Y')
holiday_2018['date'] =  pd.to_datetime(holiday_2018['date'], format='%d/%m/%Y')
holiday = pd.concat([holiday_2017, holiday_2018], ignore_index=True)
holiday['date'] = holiday.date.dt.strftime('%d/%m/%Y')
print(holiday.head())
print()
print(holiday.dtypes)

         date
0  02/01/2017
1  28/01/2017
2  30/01/2017
3  31/01/2017
4  04/04/2017

date    object
dtype: object


In [6]:
train['date_only'] = train['date'].dt.normalize()
train['date_only'] = train.date_only.dt.strftime('%d/%m/%Y')
train['holiday'] = train.date_only.isin(holiday['date'].tolist())
print(train.head())
print()
print(train.dtypes)

                  date      speed  year  ...  hour   date_only  holiday
id                                       ...                           
0  2017-01-01 00:00:00  43.002930  2017  ...     0  01/01/2017    False
1  2017-01-01 01:00:00  46.118696  2017  ...     1  01/01/2017    False
2  2017-01-01 02:00:00  44.294158  2017  ...     2  01/01/2017    False
3  2017-01-01 03:00:00  41.067468  2017  ...     3  01/01/2017    False
4  2017-01-01 04:00:00  46.448653  2017  ...     4  01/01/2017    False

[5 rows x 9 columns]

date         datetime64[ns]
speed               float64
year                  int64
month                 int64
day                   int64
weekday               int64
hour                  int64
date_only            object
holiday                bool
dtype: object


In [7]:
rainfall = pd.read_csv('/content/drive/My Drive/Python/TEMP/data/rainfall.csv')
rainfall['date'] =  pd.to_datetime(rainfall['date'], format='%d/%m/%Y')
rainfall['date_only'] = rainfall.date.dt.strftime('%d/%m/%Y')
del rainfall['date']
print(rainfall.head())
print()
print(rainfall.dtypes)

   rainfall   date_only
0       0.0  01/01/2017
1       0.0  02/01/2017
2       0.0  03/01/2017
3       0.0  04/01/2017
4       0.0  05/01/2017

rainfall     float64
date_only     object
dtype: object


In [8]:
train = pd.merge(train, rainfall, on="date_only")
print(train.head())
print()
print(train.dtypes)

                 date      speed  year  ...   date_only  holiday  rainfall
0 2017-01-01 00:00:00  43.002930  2017  ...  01/01/2017    False       0.0
1 2017-01-01 01:00:00  46.118696  2017  ...  01/01/2017    False       0.0
2 2017-01-01 02:00:00  44.294158  2017  ...  01/01/2017    False       0.0
3 2017-01-01 03:00:00  41.067468  2017  ...  01/01/2017    False       0.0
4 2017-01-01 04:00:00  46.448653  2017  ...  01/01/2017    False       0.0

[5 rows x 10 columns]

date         datetime64[ns]
speed               float64
year                  int64
month                 int64
day                   int64
weekday               int64
hour                  int64
date_only            object
holiday                bool
rainfall            float64
dtype: object


In [9]:
x = train.drop(['speed', 'date', 'date_only'], axis=1)
y = train['speed']
data_dmatrix = xgb.DMatrix(data=x,label=y)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

In [11]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.5, max_depth = 6)
xg_reg.fit(x_train,y_train)
preds = xg_reg.predict(x_test)

In [12]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE (Testing): %f" % (rmse))

RMSE (Testing): 3.324458


In [13]:
preds_train = xg_reg.predict(x_train)
rmse = np.sqrt(mean_squared_error(y_train, preds_train))
print("RMSE (Training): %f" % (rmse))

RMSE (Training): 2.199334


In [14]:
test = pd.read_csv('/content/drive/My Drive/Python/TEMP/data/test.csv', index_col=0)
test['date'] =  pd.to_datetime(test['date'], format='%d/%m/%Y %H:%M')
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['weekday'] = test['date'].dt.dayofweek
test['hour'] = test['date'].dt.hour
test['date_only'] = test['date'].dt.normalize()
test['date_only'] = test.date_only.dt.strftime('%d/%m/%Y')
test['holiday'] = test.date_only.isin(holiday['date'].tolist())
test = pd.merge(test, rainfall, on="date_only")
test = test.drop(['date', 'date_only'], axis=1)
print(test.head())
print()
print(test.info())

   year  month  day  weekday  hour  holiday  rainfall
0  2018      1    1        0     2     True       0.0
1  2018      1    1        0     5     True       0.0
2  2018      1    1        0     7     True       0.0
3  2018      1    1        0     8     True       0.0
4  2018      1    1        0    10     True       0.0

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3504 entries, 0 to 3503
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      3504 non-null   int64  
 1   month     3504 non-null   int64  
 2   day       3504 non-null   int64  
 3   weekday   3504 non-null   int64  
 4   hour      3504 non-null   int64  
 5   holiday   3504 non-null   bool   
 6   rainfall  3504 non-null   float64
dtypes: bool(1), float64(1), int64(5)
memory usage: 195.0 KB
None


In [15]:
test_preds = xg_reg.predict(test)

In [16]:
result = pd.DataFrame(test_preds)
result.to_csv('result.csv', index=False)